# Biodiversity Intactness Index Change in Phoenix, Arizona

Author: Aakriti Poudel

GitHub Repository: https://github.com/aakriti-poudel-chhetri/biodiversity-intactness-index-phoenix

# About

## Purpose
The analysis in this notebook examines how rapid urban expansion in Maricopa County, particularly within the Phoenix metropolitan area, has affected local biodiversity. By analyzing changes in the Biodiversity Intactness Index (BII) for county subdivisions between 2017 and 2020, this study aims to quantify the impact of increased development on ecosystem health and species richness. The findings will help highlight areas where urban growth has reduced biodiversity and provide insights that can inform sustainable urban planning and conservation strategies.

## Highlights

The key highlights of this analysis is to:

- Measure the area of the Phoenix subdivision with high BII (≥ 0.75) and track changes between 2017 and 2020
- Quantify the effects of rapid urban expansion on local ecosystems by analyzing changes in BII values, species richness and habitat integrity using spatial raster data
- Highlight regions within Phoenix where biodiversity has declined most, focusing on areas under ecological stress.
- Create maps showing BII distribution, areas of biodiversity loss and the Phoenix subdivision to support analysis and interpretation

## About the data

For this analysis, two primary datasets are used.

**Biodiversity Intactness Index (BII) Time Series**
This dataset is retrieved from the Microsoft Planetary Computer STAC catalog. The dataset includes raster covering the Phoenix subdivision from 2017 and 2020. These rasters measure how intact ecosystems are compared to their natural state, providing a way to track changes in biodiversity over time. The analysis is limited to the area defined by the coordinates [-112.826843, 32.974108, -111.184387, 33.863574], covering all of the Phoenix subdivision.

**Phoenix Subdivision Shapefile** 
This dataset is retrieved from Phoenix subdivision shapefile, downloaded from the Census County Subdivision files for Arizona. This shapefile provides polygon geometries for the county subdivisions, enabling spatial analysis and mapping of the BII data within the exact boundaries of Phoenix. To enhance visualization and context, shapefiles are used to show the Phoenix subdivision in relation to surrounding areas, helping to better understand the geographic distribution of biodiversity changes.

## References
Microsoft Planetary Computer. (n.d.). *Biodiversity Intactness*. Retrieved December 5, 2025, from https://planetarycomputer.microsoft.com/dataset/io-biodiversity#overview

US Census Bureau. (2025b, November 26). TIGER/Line shapefiles. Census.gov. https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html#list-tab-790442341

# Data exploration, analysis and visualization

## Import libraries

In [6]:
import pandas as pd
import geopandas as gpd

import xarray as xr
import rioxarray as rioxr
from shapely.geometry import box

import pystac_client
import planetary_computer

## Load data

In [2]:
# Load the Arizona state subdivision shapefiles
arizona = gpd.read_file('data/tl_2020_04_cousub/tl_2020_04_cousub.shp')

In [3]:
arizona

,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,04,015,91964,02582914,0401591964,Lake Havasu City,Lake Havasu City CCD,22,Z5,G4040,None,None,None,S,7493863105,50863803,+34.7686221,-113.8749805,"POLYGON ((-114.48778 34.71722, -114.48622 34.7..."
1,04,007,91277,02582913,0400791277,Fort Apache,Fort Apache CCD,22,Z5,G4040,None,None,None,S,2134058292,4388167,+33.8847663,-110.3394452,"POLYGON ((-110.78327 34.16750, -110.78276 34.1..."
2,04,003,90306,01934923,0400390306,Bisbee,Bisbee CCD,22,Z5,G4040,None,None,None,S,1610943633,3494169,+31.4580397,-110.0783372,"POLYGON ((-110.46089 31.47376, -110.46089 31.4..."
3,04,003,90408,01934925,0400390408,Bowie,Bowie CCD,22,Z5,G4040,None,None,None,S,3556140749,1110460,+32.1699052,-109.3999964,"POLYGON ((-109.76866 32.30583, -109.76787 32.3..."
4,04,003,93825,01934992,0400393825,Willcox,Willcox CCD,22,Z5,G4040,None,None,None,S,1898081614,4959613,+32.1980984,-109.8877527,"POLYGON ((-110.15218 32.17203, -110.15213 32.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,04,017,91785,01934952,0401791785,Indian Wells,Indian Wells CCD,22,Z5,G4040,None,None,None,S,3704571098,1384527,+35.3733305,-110.2639935,"POLYGON ((-110.75069 35.18569, -110.75069 35.1..."
76,04,017,93952,01934956,0401793952,Winslow-Holbrook,Winslow-Holbrook CCD,22,Z5,G4040,None,None,None,S,4574871712,11003395,+34.9152153,-110.3241699,"POLYGON ((-110.75071 35.03424, -110.75070 35.0..."
77,04,017,93213,01934981,0401793213,Snowflake,Snowflake CCD,22,Z5,G4040,None,None,None,S,4037186223,4451925,+34.4367222,-110.2461680,"POLYGON ((-110.75069 34.63665, -110.72951 34.6..."
78,04,017,91860,01934990,0401791860,Kayenta,Kayenta CCD,22,Z5,G4040,None,None,None,S,4242988226,3001923,+36.7129550,-110.3894257,"POLYGON ((-110.75070 36.42965, -110.75070 36.4..."


In [4]:
# Filter for the Phoenix subdivision shapefiles
phoenix = arizona[arizona['NAME'] == 'Phoenix'].copy()
phoenix

,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
46,04,013,92601,01934968,0401392601,Phoenix,Phoenix CCD,22,Z5,G4040,None,None,None,S,2805901266,10048820,+33.5211331,-112.0284405,"POLYGON ((-112.47014 33.55996, -112.47011 33.5..."


In [7]:
# Access Microsoft Planetary Computer STAC catalog
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier = planetary_computer.sign_inplace
)

In [9]:
# Bounding box for Phoenix
bbox = [-112.826843, 32.974108, -111.184387, 33.863574]

# Search for the BII data
search = catalog.search(
    collections = ["io-biodiversity"],
    bbox = bbox
)

In [11]:
# Retrieve items
items = search.item_collection()
print(f"There are {len(items)} items in io-biodiversity collection.")

There are 4 items in io-biodiversity collection.


In [12]:
#Check items
for item in items:
    print(f"Item ID: {item.id}, Datetime: {item.datetime}")

Item ID: bii_2020_34.74464974521749_-115.38597824385106_cog, Datetime: None
Item ID: bii_2019_34.74464974521749_-115.38597824385106_cog, Datetime: None
Item ID: bii_2018_34.74464974521749_-115.38597824385106_cog, Datetime: None
Item ID: bii_2017_34.74464974521749_-115.38597824385106_cog, Datetime: None
